# Classify Ampules


## Getting started

Erstmal müssen wir unser Repository in unsere Colab Instanz laden.

In [1]:
!git clone https://github.com/ju-leon/ClassifyAmpules.git
%cd ClassifyAmpules

Cloning into 'ClassifyAmpules'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 137 (delta 79), reused 92 (delta 40), pack-reused 0
Receiving objects: 100% (137/137), 3.50 MiB | 3.20 MiB/s, done.
Resolving deltas: 100% (79/79), done.
/content/ClassifyAmpules


### Google Drive

Wir verwenden **Google Drive** um Daten zu laden und zu speichern.
Zum Einbinden von Google Drive dem Link unten folgen:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Updating

Um Änderungen in unserem Repositiory sichbar zu machen muss es neu geladen werden:

In [ ]:
!git pull

## Setup

Hier setzen wir unser Dateisystem auf.

**Source Path:** Pfad zu den Rohdaten

**Cache Path:** Pfad in ein leeren Ordner. Hier werden Bilder nach der ersten Preprocessing Schritt gespeichert.

**Out Path:** Pfad in ein leeren Ordner. Hier werden die fertig verarbeiteten Bilder gespeichert.

In [3]:
source_path = "/content/drive/My Drive/Hackaton/PIM_Dataset"
cache_path = "/content/drive/My Drive/Hackaton/PIM_combi"
out_path =  "/content/drive/My Drive/Hackaton/combined"

source_path_good = "'" + source_path + "/good_piece/" + "'"
source_path_bad = "'" + source_path + "/bad_piece" + "'"

cache_path_good = "'" + cache_path + "/good_piece/" + "'"
cache_path_bad = "'" + cache_path + "/bad_piece/" + "'"

out_path_good = "'" + out_path + "/good_piece/" + "'"
out_path_bad = "'" + out_path + "/bad_piece/" + "'"
out_path = "'" + out_path + "'"

Falls die (leeren) Ordner noch nicht existieren erstellen wir sie...

In [4]:
%mkdir -p $cache_path_good $cache_path_bad $out_path_good $out_path_bad

# Preprocessing

![Raw Data](https://github.com/ju-leon/ClassifyAmpules/blob/master/docs/metal226_3.bmp?raw=1)

Als erstes müssen wir die Datenmenge reduzieren.

Wir verwenden einen **min Filter** der Größe 9 bevor wir runter skalieren. 
Dadurch werden Abweichungen von dem Hintergrund wie Metallteile hervorgehoben und vergrößert. Das verhindert, das diese Merkmale beim runterskalieren verloren gehen.
Hier sieht man, dass das Metallstück rechts unten deutlich größer erscheint und somit beim runterskalieren noch deutlich besser sichtbar ist.


![Raw Data](https://github.com/ju-leon/ClassifyAmpules/blob/master/docs/after_preprocess.png?raw=1)

Danach scheiden wir den wichtigen Bildbereich aus.
Mit mehr Zeit könnte man dieses Ausschneiden intelligent machen, das also immer genau die Ampullen ausgeschnitten werden.
Das würde die Datenmenge deutlich reduzieren, die Variation zwischen den Bildern minimieren, und damit höchstwahrscheinlich das Klassifikationsergebniss um ein Vielfaches verbessern.

In [ ]:
!python3 preprocess.py $source_path_good $cache_path_good
!python3 preprocess.py $source_path_bad $cache_path_bad

# Combining

Für jedes Sample werden 3 Bilder angegeben. 
Um ein Overfitting zu vermeiden, müssen diese Bilder kombiniert werden, sonst könnte das Netz allein anhand der Ausrichtung der Ampullen erkennen ob sie Verunreinigt sind. Das würde die Metriken bei Training nutzlos machen.

Deshab legen wir alle 3 Bilder übereinander. Wir nehmen für jedes Pxxel das mit dem dunkelsten Wert.
Dadurch bleiben wichtige Merkmale wie Metallstücke erhalten, doppelte Info wie Ränder verschwinden.

![Raw Data](https://github.com/ju-leon/ClassifyAmpules/blob/master/docs/metal226.png?raw=1)

In [ ]:
!python3 combine.py $cache_path_good $out_path_good
!python3 combine.py $cache_path_bad $out_path_bad

# Filter by bad piece

Hier werden die schlechten Bilder in die einzelnen Klassen aufgeteilt. Das wird nur dann benötigt, wenn man auch noch klassifizieren möchte welcher Fehler auftritt.

In [ ]:
from shutil import copyfile
import os
path = "/content/drive/My Drive/Hackaton/combined/good_piece/"

valid_images = ".png"
for f in os.listdir(path):
    if f.endswith(valid_images):
        copyfile(path + f, "/content/drive/My Drive/Hackaton/all_classes/good/" + f)


# Training

Jetzt können wir das Netz trainieren. 


Wir verwenden Tensorflow und die high-level API Keras zum Trainieren eines neuronalen Netzes.
Bei unserer Architektur, setzen wir angelehnt an Imagenet (vgl. https://arxiv.org/abs/1409.0575)
auf ein CNN (Convolutional Neural Network) das mehrere Convolutional Blocks (Convolutional Layer + Max Pooling Layer) beinhaltet, die mit Relu aktiviert werden. Darauf folgt ein Dropout und darauf 3 Fully Connected Layer um zu klassifizieren. Der finale Output wird durch einen Softmax-Layer erzeugt. 

Dieser erzeugt Pseudowahrscheinlichkeiten für die Zugehörigkeit eines gebenen Sample zur jeweiligen Klasse (Good Piece, Bad Piece).
Diese Architektur hat sich zum Klassifizieren von Bildern bewährt, da Features durch Convolutional Layer gelernt werden und dann durch die Fully Connected Layer klassifiziert werden können.

Wir haben auch Tests mit einem Netzwerk das auf metal, gasket, charred, good klassifiziert gemacht. Dabei haben wir ein accurcy von 75% erreicht, wobei die meisten Fehler innerhalb der "bad-Klassen" erfolgt sind. die Fehler wären also nicht besonders schlimm, da nur der grund für die Ablehnung falsch klassifiziert wurde, aber nur selten fälschlicherweise eine defekte Charge als gut klassifiziert wurde. 
Dennoch ist insgesamt die Fehlerhäufigkeit höher, was auch an der sehr geringen Datenmenge pro Klasse liegen könnte wenn man die "bad_piece" Klasse noch weiter in 3 Unterklassen aufteilt.


Unsere Daten werden geshuffelt und in Test und Validierungsdaten aufgeteilt (Verhältnis 70%:30%).
Wir verwenden ADAM (vgl. https://arxiv.org/abs/1412.6980) als Optimierungsalgorithmus und validieren die Optimierung nach jeder Trainingsepoche mit unserem Validierungsdatensatz. Verbessert sich die Genauigkeit des Netzes wird dieses gespeichert. Am Ende des Training werden mit dem besten Netz auf dem Validierungsdatensatz die Confusion Matrix und Metriken wie f1-score, precision, ... erzeugt.

## Anmerkung zu Hyperparameteroptimierung
Parameter wie Lernrate, Anzahl der Schichten, Größe der Schichten, Größe der Convultional Filter usw. können durch Systematische Ansätze der Hyperparameteroptimierung (wie z.B Neural Architecture Search) immer weiter Optimiert werden. Solche Ansätze sprengen leider den Zeitlichen Rahmen eines Hackathons, führen aber, auf Grundlage der von uns geschaffenen Architektur, zu exzellenten Ergebnissen. Die weiter Optimierung wäre für den Produktiveinsatz wünschenswert.


In [ ]:
!python3 train.py $out_path

Using TensorFlow backend.
2020-07-03 14:37:20.562516: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Found 892 images belonging to 2 classes.
Found 382 images belonging to 2 classes.
2020-07-03 14:37:23.272350: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-03 14:37:23.315794: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-03 14:37:23.316575: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-07-03 14:37:23.316628: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully ope

## Ergebniss

Nach dem Training über 100 Epochen hat unser Model folgende Ergebnisse

![Raw Data](docs/result.jpeg)

## Web UI


Zum Einfachen testen haben wir zudem eine Web UI begonnen, die wie folgt aussieht:


![Raw Data](docs/edit.gif)
![Raw Data](docs/webui.jpeg)